## Pandas (Combining Datasets - Merge and Join)
### Example - US States Data

Ejemplo de uso de Merge y Joins combinando datos de distintas fuentes.

Queremos ordenar los estados y territorios de USA en función de la densidad de población en 2010

In [2]:
import numpy as np
import pandas as pd 

Descargamos los ficheros csv de trabajo
* state-population.csv
* state-areas.csv
* state-abbrevs.csv

In [3]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0   159k      0 --:--:-- --:--:-- --:--:--  159k


In [4]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   4489      0 --:--:-- --:--:-- --:--:--  4489


In [5]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   2121      0 --:--:-- --:--:-- --:--:--  2116


#### Cargamos los ficheros en unos df

En primer lugar, necesitamos cargar los datos descargados en unos df, para poder manipular la información. Para ello utilizaremos la función read_csv(), ya que los datos descargados sabemos que están en csv (habría que haber hecho un paso previo para ver si los datos realmente están en csv)

In [6]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

#### Echamos un vistazo a los datos

Una vez cargados los datos, lo primero que hacemos es hechar un vistazo a una muestra de estos, para ver que pinta tienen

In [12]:
# población
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [13]:
# áreas
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [14]:
# abreviaturas
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


#### Manipulamos los datos

Una vez sabemos como están estructurados los datos, procedemos a 
realizar las modificaciones sobre ellos para poder extraer la información
que deseamos conocer

In [20]:
# Hacemos un join entre el df de población y el de abreviaturas, para
# obtener el nombre de los estados, y que podamos luego cruzarlo
# con el df de las áreas

# Usamos left_on y right_on ya que las columnas tienen distintos nombres
# Usamos how "outer" para asegurarnos de no perder datos debido 
# a errores en la asignación de los estados en la tabla de poblaciones

merged = pd.merge(pop, abbrevs, left_on="state/region", right_on="abbreviation", how='outer')

In [21]:
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [22]:
# Eliminamos la columna redundante de abreviatura, ya que la tenemos
# duplicada
merged = merged.drop('abbreviation', axis=1)

In [23]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


#### Buscamos valores NaN

In [30]:
# Vamos a verificar que después del cruce no tenemos valores nulos
merged.isnull().head()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False


Nos devuelve un df booleano donde por cada elemento se evalua si 
existe algun campo con NA

In [31]:
# Verificamos si hay alguna columna que tenga valores nulos
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

### Población con valores NaN

Parece que tenemos valores nulos en las columnas de población y estado

In [35]:
# filtramos aquellos registros del df que tienen valores nulos
# en la columna de población, y mostramos un ejemplo del df resultante
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [36]:
m1 = merged[merged['population'].isnull()]

In [37]:
# determinamos a qué estados pertenecen estos valores NA
# con unique, nos saca los valores distintos de la columna
m1['state/region'].unique()

array(['PR'], dtype=object)

In [38]:
# determinamos de qué periodo de años no tenemos datos
m1['year'].values.max()

1999

In [39]:
m1['year'].values.min()

1990

Como lo que vamos a necesitar son los datos de 2010, no nos haría falta hacer nada sobre estos datos, ya que luego filtraremos este año para hacer los cálculos

### 'State' con valores NaN

In [41]:
# Vamos a sacar los estados de los que no tenemos abreviatura también,
# que es otra de las columnas de las que tenemos NaN
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [47]:
# filtra dentro del data frame
# .loc[filtro, columnas] : indexador para indicar que queremos filtrar
#      filtro: merged['state'].isnull() todas las filas cuyo valor de
#              'state' es nulo
#       columnas:  devuelve la/s columna/s
m2 = merged.loc[merged['state'].isnull(), 'state/region']

In [48]:
m2.head()

2448    PR
2449    PR
2450    PR
2451    PR
2452    PR
Name: state/region, dtype: object

In [49]:
# Sacamos aquellos estados que están afectados por no tener
# la abreviatura
m2.unique()

array(['PR', 'USA'], dtype=object)

Podemos inferir que se trata del estado de "Puerto Rico" y que 'USA'
se corresponde con unos valores globales que hacen referencia a todo
Estados Unidos. Esto podemos arreglarlo fácilmente, asignando el valor de estado a estas filas.

In [50]:
# filtramos por la columna 'state/region' y actualizamos la columna
# state
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [53]:
merged.loc[merged['state/region'] == 'PR', 'state'].head()

2448    Puerto Rico
2449    Puerto Rico
2450    Puerto Rico
2451    Puerto Rico
2452    Puerto Rico
Name: state, dtype: object

In [54]:
# Verificamos si ya no tenemos valores nulos en el campo de estado
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

###  Unimos el df de población (merged) con el de áreas

Sabiendo que ya no tenemos valores NaN en el campo de estado, podemos 
realizar ya un join entre el df de población y el de áreas

In [60]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [56]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [62]:
# hacemos un left join, para mantener los datos de población
# independientemente si encontramos datos del área para la
# fila, y utilizamos como columna clave 'state'
final = pd.merge(merged, areas, on='state', how='left')

In [63]:
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


#### Verificamos NaN en el nuevo df

In [64]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Seguimos teniendo NaN values en la columna de población y también
tenemos en la columna de 'area'

#### Analizamos los NaN en la columna de área

In [67]:
# Echamos un vistazo a las filas con registros NaN
final.loc[final['area (sq. mi)'].isnull()].head()

,state/region,ages,year,population,state,area (sq. mi)
2496,USA,under18,1990,64218512.0,United States,NaN
2497,USA,total,1990,249622814.0,United States,NaN
2498,USA,total,1991,252980942.0,United States,NaN
2499,USA,under18,1991,65313018.0,United States,NaN
2500,USA,under18,1992,66509177.0,United States,NaN


Parece que se tratan de los registros totales de USA, vamos a ver 
si es así

In [69]:
# recuperamos los valores distintos del campo 'state/region'
final.loc[final['area (sq. mi)'].isnull(), 'state/region'].unique()

array(['USA'], dtype=object)

Podríamos utilizar la suma de todos los estados para calcular el área de USA pero como no nos interesa para lo que estamos analizando, directamente vamos a eliminar esta información, tanto de USA, como los NaN de población correspondiente a los datos de Puerto Rico de años anteriores al 2010 que vimos anteriormente

In [70]:
# eliminamos las filas con valores nulos. Usamos inplace para que lo
# haga directamente sobre el df
final.dropna(inplace=True)

In [71]:
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [73]:
# verificamos nulos de nuevo
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

Ya disponemos de un df limpio, sin valores nulos!

### Filtrando la información

Nos interesa quedarnos ahora con el año 2010 y con los datos totales

In [75]:
data2010 = final[(final['year'] == 2010) & (final['ages'] == 'total')]

In [76]:
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Ya tenemos la información que necesitamos

### Calculando la densidad poblacional

In [78]:
# establecemos como índice el campo de estado, ya que vamos a crear
# un nuevo df con el cálculo de la densidad poblacional y si ponemos
# como índice el estado, ya arrastramos este y no tenemos que realizar
# manipulaciones adicionales
data2010 = data2010.set_index('state')

In [80]:
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [81]:
# creamos un nuevo df con el cálculo
density = data2010['population'] / data2010['area (sq. mi)']

In [82]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

## Ranking!!

In [83]:
# ordenamos el df en orden descendente y actualizamos el df
density.sort_values(ascending=False, inplace=True)

In [84]:
# más pobladas
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [85]:
# menos pobladas
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64